In [ ]:
!pip install git+https://github.com/maxghenis/microdf.git

In [270]:
import pandas as pd
import plotly.express as px
import microdf as mdf
import numpy as np

In [271]:
person_raw = pd.read_csv('https://github.com/MaxGhenis/datarepo/raw/master/pppub19.csv.gz',
                         usecols={'MARSUPWT', 'SPM_ID', 'SPM_RESOURCES', 'SPM_POVTHRESHOLD',
                         'A_AGE', 'PEDISDRS', 'PEDISEAR', 'PEDISEYE', 'PEDISOUT', 'PEDISPHY', 'PEDISREM'})

In [272]:
person = person_raw.copy(deep = True) 
person.columns = person.columns.str.lower()
person['weight'] = person.marsupwt / 100

In [273]:
del person['marsupwt']

In [274]:
## find column that says # of people in SPM Unit
## filter only adults

person['child'] = person.a_age < 18
person['adult'] = person.a_age >= 18
ages = person.groupby('spm_id')[['child', 'adult']].sum() 
ages.columns = ['total_children', 'total_adults'] 
person = person.merge(ages, left_on='spm_id', right_index = True)
person['total_people'] = person.total_adults + person.total_children
person['disabled'] = (person.pedisdrs == 1) | (person.pedisear == 1) | (person.pediseye == 1) | (person.pedisout == 1) | (person.pedisphy == 1) | (person.pedisrem == 1)

In [275]:
total_disabled = (person.weight * person.disabled).sum()
total_disabled/1e6

30.338139509999994

In [276]:
columns_list = {'pedisdrs': 'Self-care', 'pedisear': 'Hearing', 'pediseye': 'Vision', 'pedisout': 'Mobility', 'pedisphy': 'Physical', 'pedisrem': 'Cognitive', 'disabled': 'Any'}

In [277]:
def disabled_pov(column, monthly_ubi=0):
    
    target_persons = person[person[column]==1].copy(deep=True)
    target_persons['total_ubi'] = person.total_people * monthly_ubi * 12
    target_persons['new_spm_resources'] = target_persons.total_ubi + target_persons.spm_resources
    target_persons['poor'] = target_persons.new_spm_resources < target_persons.spm_povthreshold
    population_target = target_persons.weight.sum()
    total_target_poor = (target_persons.poor * target_persons.weight).sum()
    total_pov_rate = total_target_poor / population_target

    return (total_pov_rate * 100).round(3)

def disabled_pov_row(row):
    return disabled_pov(row.column, row.monthly_ubi)

In [ ]:
summary = mdf.cartesian_product({'column':columns_list,
                       'monthly_ubi': np.arange(0,1001,100)})

summary['poverty_rate'] = summary.apply(disabled_pov_row, axis=1)
pd.set_option("display.max_rows", None, "display.max_columns", None)
summary['disability_type'] = summary.column.map(columns_list)

In [ ]:
#TO DO

# see if the numbers will auto display
# colors are changing back to normal when bar is moved

colors = ['darkblue', 'darkred', 'orange', 'mediumseagreen', 'gold', 'indianred']

fig = px.bar(summary, x = 'disability_type', y='poverty_rate',
             animation_frame='monthly_ubi', range_y=[0,25],
             labels={
                     "poverty_rate": "SPM Poverty Rate",
                     "disability_type": "Disability Type",
                     "monthly_ubi": "Monthly UBI"
                 },
                title="The effect of a UBI on poverty among people with disabilites")
fig.update_traces(marker_color=colors, opacity=1.0)

fig.show()